In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras import layers, models
from keras.models import Sequential

from tensorflow.keras.callbacks import EarlyStopping


## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load("tf_flowers", split=["train[:70%]", "train[30%:]"],
                                                             batch_size=-1,as_supervised=True,)  # Include labels



## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)


## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)
print(base_model.summary())


model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

print(model.summary())

model.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=5, validation_split=0.2, batch_size=32, callbacks=[es])

test_pred=model.predict(test_ds)
score= model.evaluate(test_ds,test_labels, batch_size=128, verbose=0)
print(score)
